In [1]:
%pip install mlflow


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import pandas as pd

In [5]:
mlflow.set_tracking_uri("databricks")


In [13]:
with mlflow.start_run() : 
    rating_df=pd.read_csv("../data/rating.csv")
    domain_locations_df = pd.read_csv("../data/domains_location.csv")
    traffic_data_df = pd.read_csv("../data/traffic.csv")

    # Drop duplicates
    rating_df.drop_duplicates(inplace=True)
    domain_locations_df.drop_duplicates(inplace=True)
    traffic_data_df.drop_duplicates(inplace=True)

    # Drop unnecessary columns
    if 'source_id' in rating_df.columns:
        rating_df.drop(columns=['source_id'], inplace=True)

    if 'url_to_image' in rating_df.columns:
        rating_df.drop(columns=['url_to_image'], inplace=True)

    # Drop rows with missing values
    rating_df = rating_df.dropna(axis=0)
    domain_locations_df = domain_locations_df.dropna(axis=0)

    merge_df=pd.merge(rating_df, domain_locations_df ,left_on='source_name', right_on ='SourceCommonName' ,how ='left')
    merge_df=pd.merge(merge_df, traffic_data_df , left_on ='source_name' ,right_on='Domain' ,how ='left')

    # Train topic modeling model
    tfidf_vectorizer = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf_vectorizer.fit_transform(merge_df['clean_content'])
    num_clusters = 5  # Adjust as needed
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(tfidf_matrix)

    # Log parameters and metrics
    mlflow.log_param("num_clusters", num_clusters)

    # Save trained models and artifacts
    mlflow.sklearn.log_model(kmeans, "topic_model")
    mlflow.sklearn.log_model(tfidf_vectorizer, "tfidf_vectorizer")


InvalidConfigurationError: You haven't configured the CLI yet! Please configure by entering `C:\Users\dell\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py configure`